In [9]:
import inst_utils
reload(inst_utils)
from labjack import ljm
import time
import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

In [10]:
version = '180514_1_Warm_up'
folder_path = 'data/%s'%(version)
file_path = folder_path + '/power_log.txt'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
with open(file_path, 'w') as f_t:
    f_t.write('ctime\t Voltage1\t Current1\t Power_40K\t Voltage2\t Current2\t Power_4K\n')
    f_t.write('seconds\t Volts\t Amps\t Watts\t Volts\t Amps\t Watts\n')

In [24]:
#array of [40K_power, 4K_power]
pwr_arr = np.array([[0., 8.],])
'''
pwr_arr = np.array([[0., 0.], [0., 0.5], [0., 1.0], [0., 1.5], 
                     [0., 0.], [0., 0.], 
                     [4., 0.], [4., 0.5], [4., 1.0], [4., 1.5], 
                     [0., 0.], [0., 0.],
                     [8., 0.], [8., 0.5], [8., 1.0], [8., 1.5],
                   ])
plt.figure()
plt.plot(pwr_arr[:, 0], pwr_arr[:, 1], 'o-')
plt.xlabel('40K Stage Heating (W)')
plt.ylabel('4K Stage Heating (W)')
'''

"\npwr_arr = np.array([[0., 0.], [0., 0.5], [0., 1.0], [0., 1.5], \n                     [0., 0.], [0., 0.], \n                     [4., 0.], [4., 0.5], [4., 1.0], [4., 1.5], \n                     [0., 0.], [0., 0.],\n                     [8., 0.], [8., 0.5], [8., 1.0], [8., 1.5],\n                   ])\nplt.figure()\nplt.plot(pwr_arr[:, 0], pwr_arr[:, 1], 'o-')\nplt.xlabel('40K Stage Heating (W)')\nplt.ylabel('4K Stage Heating (W)')\n"

In [22]:
ps_4K = inst_utils.Agilent_E3647a_PS()
lj = ljm.openS("T7", "usb", "CryoTester_ljT7")
multi_volt = inst_utils.HP_3478A_multimeter(gpib_address=3)
multi_curr = inst_utils.HP_3478A_multimeter(gpib_address=26)

In [25]:
duration = 3600 #s
#pwr = 0 #W
res1 = 12. #Ohms
res2 = 12. #Ohms
pause = 5 #seconds

v_factor = 50./5.0 #0-5V analog represents 0-50V
i_factor = 24./5.0 #0-5V analog represents 0-24A

ps_switch = 'FIO0'
v_ctrl = 'DAC0'

for pwrs in pwr_arr:
    
    voltage = np.sqrt(res1 * pwrs[0])
    ljm.eWriteName(lj, ps_switch, 0) #turn on the power supply
    ljm.eWriteName(lj, v_ctrl, voltage/v_factor)
    
    ps_4K.set_current(np.sqrt(pwrs[1]/res2), ch=1)
    to = time.time()
    time.sleep(pause)
    volt1_t = multi_volt.meas_voltage()*v_factor
    curr1_t = multi_curr.meas_voltage()*i_factor
    volt2_t = ps_4K.meas_voltage(ch=1)
    curr2_t = ps_4K.meas_current(ch=1)
    print 'voltage, current (40K):',volt1_t, curr1_t, volt1_t * curr1_t,\
          '; voltage, current (4K):', volt2_t, curr2_t, volt2_t * curr2_t
    while (time.time() - to) < duration:
        time_t = time.time()
        volt1_t = multi_volt.meas_voltage()*v_factor
        curr1_t = multi_curr.meas_voltage()*i_factor
        power1_t = volt1_t * curr1_t
        volt2_t = ps_4K.meas_voltage(ch=1)
        curr2_t = ps_4K.meas_current(ch=1)
        power2_t = volt2_t * curr2_t
        line_t = '%.3f\t %f\t %f %f\t %f\t %f %f\n'%(time_t, volt1_t, curr1_t, power1_t,
                                                    volt2_t, curr2_t, power2_t)
        with open(file_path, 'a') as f_t:
            f_t.write(line_t)
print 'Heating script is finished. Power is to be turned to zero.'

# Turn the power to zero
pwr = 0. #W
voltage = np.sqrt(res1 * pwr)
ljm.eWriteName(lj, v_ctrl, voltage/v_factor)

ps_4K.set_current(np.sqrt(pwr/res2), ch=1)
to = time.time()
time.sleep(pause)
volt1_t = multi_volt.meas_voltage()*v_factor
curr1_t = multi_curr.meas_voltage()*i_factor
volt2_t = ps_4K.meas_voltage(ch=1)
curr2_t = ps_4K.meas_current(ch=1)
print 'voltage, current (40K):',volt1_t, curr1_t, volt1_t * curr1_t,\
      '; voltage, current (4K):', volt2_t, curr2_t, volt2_t * curr2_t
while True:
    time_t = time.time()
    volt1_t = multi_volt.meas_voltage()*v_factor
    curr1_t = multi_curr.meas_voltage()*i_factor
    power1_t = volt1_t * curr1_t
    volt2_t = ps_4K.meas_voltage(ch=1)
    curr2_t = ps_4K.meas_current(ch=1)
    power2_t = volt2_t * curr2_t
    line_t = '%.3f\t %f\t %f %f\t %f\t %f %f\n'%(time_t, volt1_t, curr1_t, power1_t,
                                                volt2_t, curr2_t, power2_t)
    with open(file_path, 'a') as f_t:
        f_t.write(line_t)

voltage, current (40K): 0.08715 0.0401856 0.00350217504 ; voltage, current (4K): 10.01809 0.8244007 8.25892040866


KeyboardInterrupt: 

In [26]:
ljm.eWriteName(lj, ps_switch, 1)
ljm.close(lj)
multi_volt.close()
multi_curr.close()
ps_4K.close()

# Two outputs from Agilent Power Supply

In [24]:
ps_4K = inst_utils.Agilent_E3647a_PS()
#ps.device.write('DISP ON')

duration = 3600 #s
#pwr = 0 #W
res1 = 12. #Ohms
res2 = 12. #Ohms
pause = 2 #seconds

for pwrs in pwr_arr:
    ps.set_current(np.sqrt(pwrs[0]/res1), ch=1)
    ps.set_current(np.sqrt(pwrs[1]/res2), ch=2)
    to = time.time()
    time.sleep(pause)
    volt1_t = ps.meas_voltage(ch=1)
    curr1_t = ps.meas_current(ch=1)
    volt2_t = ps.meas_voltage(ch=2)
    curr2_t = ps.meas_current(ch=2)
    print volt1_t, curr1_t, volt2_t, curr2_t
    while (time.time() - to) < duration:
        time_t = time.time()
        volt1_t = ps.meas_voltage(ch=1)
        curr1_t = ps.meas_current(ch=1)
        power1_t = volt1_t * curr1_t
        volt2_t = ps.meas_voltage(ch=2)
        curr2_t = ps.meas_current(ch=2)
        power2_t = volt2_t * curr2_t
        line_t = '%.3f\t %f\t %f %f\t %f\t %f %f\n'%(time_t, volt1_t, curr1_t, power1_t,
                                                    volt2_t, curr2_t, power2_t)
        with open(file_path, 'a') as f_t:
            f_t.write(line_t)
pwr = 0.
ps.set_current(np.sqrt(pwr/res1), ch=1)
ps.set_current(np.sqrt(pwr/res2), ch=2)
to = time.time()
time.sleep(pause)
volt1_t = ps.meas_voltage(ch=1)
curr1_t = ps.meas_current(ch=1)
volt2_t = ps.meas_voltage(ch=2)
curr2_t = ps.meas_current(ch=2)
print volt1_t, curr1_t, volt2_t, curr2_t
while True:
    time_t = time.time()
    volt1_t = ps.meas_voltage(ch=1)
    curr1_t = ps.meas_current(ch=1)
    power1_t = volt1_t * curr1_t
    volt2_t = ps.meas_voltage(ch=2)
    curr2_t = ps.meas_current(ch=2)
    power2_t = volt2_t * curr2_t
    line_t = '%.3f\t %f\t %f %f\t %f\t %f %f\n'%(time_t, volt1_t, curr1_t, power1_t,
                                                volt2_t, curr2_t, power2_t)
    with open(file_path, 'a') as f_t:
        f_t.write(line_t)

9.85433 0.8244125 9.810174 0.8239621


KeyboardInterrupt: 

In [25]:
ps.close()